In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [45]:
def fetch_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    return data

def calculate_returns(data):
    returns = data.pct_change()*100
    return returns

def calculate_risk_reward_ratio(returns):
    mean_return = returns.mean()
    std_return = returns.std()
    risk_reward_ratio = mean_return / std_return
    return risk_reward_ratio

def calculate_correlation(returns):
    correlation_matrix = returns.corr()
    return correlation_matrix

def find_best_pairs(risk_reward_ratios, correlation_matrix):
    # Find top 5 stocks with highest risk/reward ratios
    top_5_stocks = risk_reward_ratios.nlargest(5).index.tolist()
    best_pairs = {}
       
    # Find least correlated stock for each of the top 5 stocks
    for stock in top_5_stocks:
        least_correlated_stock = correlation_matrix[stock].idxmin()
        best_pairs[stock] = least_correlated_stock

    return best_pairs


def calculate_moving_averages(data):
    short_rolling = data.rolling(window=20).mean()
    medium_rolling = data.rolling(window=50).mean()
    long_rolling = data.rolling(window=200).mean()
    return short_rolling, medium_rolling, long_rolling

In [46]:
def plot_data(data, medium_rolling, long_rolling, ticker):
    fig, ax1 = plt.subplots(figsize=(12, 6))

    ax1.plot(data, label='Price', color='blue')
    ax1.plot(medium_rolling, label='50-Day SMA', color='green')
    ax1.plot(long_rolling, label='200-Day SMA', color='red')
    ax1.set_ylabel('Price')
    
    ax1.set_title(f'Stock: {ticker}')  # Set the title with the stock ticker symbol

    # Calculate profit
    buy_signal = (medium_rolling > long_rolling) & (medium_rolling.shift(1) < long_rolling.shift(1))
    sell_signal = (medium_rolling < long_rolling) & (medium_rolling.shift(1) > long_rolling.shift(1))
    
    buy_prices = data[buy_signal]
    sell_prices = data[sell_signal]
    
    profit = 0
    while not buy_prices.empty and not sell_prices.empty:
        buy_index, buy_price = buy_prices.index[0], buy_prices.iloc[0]
        sell_index, sell_price = sell_prices.index[0], sell_prices.iloc[0]
        
        if sell_index > buy_index:
            profit += (sell_price - buy_price)
            print(f"Profit: {sell_price - buy_price}")
            buy_prices = buy_prices.drop(buy_index)
            sell_prices = sell_prices.drop(sell_index)
        elif buy_index > sell_index:
            profit += (sell_price - buy_price)
            print(f"Profit: {sell_price - buy_price}")
            sell_prices = sell_prices.drop(sell_index)
            buy_prices = buy_prices.drop(buy_index)
        else:
            break
    # Calculate profit for last trade without pair
    if not buy_prices.empty:
        # Last trade is a buy without a pair
        last_buy_price = buy_prices.iloc[0]
        current_price = data.iloc[-1]  # Current price is the last price in the data
        profit += (current_price - last_buy_price)
        print(f"Profit for Last Buy Trade without Pair: {current_price - last_buy_price}")
    elif not sell_prices.empty:
        # Last trade is a sell without a pair
        last_sell_price = sell_prices.iloc[0]
        current_price = data.iloc[-1]  # Current price is the last price in the data
        profit += (last_sell_price - current_price)
        print(f"Profit for Last Sell Trade without Pair: {last_sell_price - current_price}")

        
    print(f"Total Profit: {profit}")
        
    # Add buy and sell signals based on moving average crossovers
    ax1.plot(medium_rolling.index[buy_signal], medium_rolling[buy_signal], '^', markersize=10, color='g', lw=0, label='Buy Signal')
    ax1.plot(medium_rolling.index[sell_signal], medium_rolling[sell_signal], 'v', markersize=10, color='r', lw=0, label='Sell Signal')
        
    # Manually create legend for buy and sell signals
    ax1.legend(loc='upper left')
    
    plt.show()


In [47]:
def plot_correlation(correlation_matrix):
    plt.figure(figsize=(16, 16))
    plt.imshow(correlation_matrix, cmap='coolwarm', interpolation='none')
    plt.colorbar()
    tickers = correlation_matrix.columns.tolist()
    plt.xticks(range(len(tickers)), tickers, rotation=90)
    plt.yticks(range(len(tickers)), tickers)
    plt.title('Correlation Matrix')
    for i in range(len(tickers)):
        for j in range(len(tickers)):
            plt.text(j, i, "{:.2f}".format(correlation_matrix.values[i, j]), ha='center', va='center', color='black')
    plt.show()

In [48]:
def plot_risk_reward(risk_reward_ratios, tickers):
    if isinstance(risk_reward_ratios, (float, np.float64)):
        # If risk_reward_ratios is a single float value, convert it to a list
        risk_reward_ratios = [risk_reward_ratios]
        tickers = [tickers]  # Assuming tickers is also a single value
        
    plt.figure(figsize=(10, 6))
    plt.scatter(range(len(risk_reward_ratios)), risk_reward_ratios, color='skyblue')
    plt.title('Risk/Reward Ratios')
    plt.xlabel('Stocks')
    plt.ylabel('Risk/Reward Ratio')
    plt.xticks(range(len(risk_reward_ratios)), range(1, len(risk_reward_ratios) + 1))
    
    # Add labels for each data point
    for i, ratio in enumerate(risk_reward_ratios):
        plt.text(i, ratio, f'{tickers[i]}', ha='center', va='bottom', fontsize=8)
    
    plt.show()


In [ ]:
def main():
    tickers = input("Enter the tickers of stocks you want to analyze (separated by commas): ").split(',')   
    start_date = '2013-04-29'
    end_date = '2024-04-29'
    
    # Fetch data
    data = fetch_data(tickers, start_date, end_date)
    
    # Calculate returns
    returns = calculate_returns(data)
    
    # Calculate risk/reward ratios
    risk_reward_ratios = calculate_risk_reward_ratio(returns)
    
    # Plot risk/reward ratios
    plot_risk_reward(risk_reward_ratios, tickers)
    
    # Calculate correlation matrix
    correlation_matrix = calculate_correlation(returns)
    
    # Plot correlation matrix
    plot_correlation(correlation_matrix)
    
    # Find best pairs
    best_pairs = find_best_pairs(risk_reward_ratios, correlation_matrix)
    
    all_stocks = set(best_pairs.keys())
    for stock, least_correlated_stock in best_pairs.items():
        all_stocks.add(least_correlated_stock)
    
    # Calculate and plot moving averages for all stocks in the set
    for ticker in all_stocks:
        short_rolling, medium_rolling, long_rolling = calculate_moving_averages(data[ticker])
        plot_data(data[ticker], short_rolling, medium_rolling, ticker)


if __name__ == "__main__":
    main()